# stack limitations

- Everything the current functions’ stack frame is destroyed upon return
•    can’t create something on the stack and return a pointer to it

• The stack is limited in size
    • can’t put large arrays on the stack
    • very deep recursion can overflow the stack

• Once created, a stack-allocated array’s size can’t be changed

• Variable-length arrays may or may not be possible on the stack

• an optional part of the C11 and later standards

## game of nim

Start with piles of stones
• Players take turns taking any positive
number of the remaining stones from
a single pile
• Last move wins

In [ ]:
unsigned int *read_game(size_t *num_piles)
{
    fscanf(stdin, "%zu", num_piles);
    unsigned int pile_size[*num_piles];
    // read each pile size
    size_t i = 0;
    while (i < *num_piles && scanf("%u", &pile_size[i]) == 1)
        i++;
    // fill in any missing values with 0
    while (i < *num_piles)
    {
        pile_size[i] = 0;
        i++;
    }
    return pile_size;
}

pointer to a pointer! dangling pointer :(

# the heap
space on the stack is automatically allocated and freed as functions begin/end execution

you can point to a place in the heap, and the OS will not touch it!

note that the OS trusts you to free this manually

how to use it:

to allocate space on the heap:

In [ ]:
void *malloc(size_t size);

to free space on the heap:

In [ ]:
void free(void *ptr);

using that, free up space on the heap allocated by malloc

note malloc's use of size_t. only positive sizes allowed.

In [ ]:
#include <stdlib.h>// for malloc and free
#include <stdio.h>
int main() {

    int *pi;
    // this results in sigkill (OS kills your program)
    // while (1) {
    //     pi = malloc(sizeof(int));
    // }

    pi = malloc(sizeof(int));
    if (pi == NULL) return 1; // if malloc fails pi will be null

    // -------types of errors--------//
    
    // now imagine you cross out the address like so:
    // pi = NULL;
    // free (pi); // nothing happens... you can free nothing
    // but valgrind will return a leak! it will say 4 bytes were lost (which is the size of an integer, so it checks out)

    // ------------------------------------//

    // imagine you dereference a null value
    // pi = NULL;
    // *pi = 5; // *NULL (you are dereferencing a NULL!) // this line leads to a segmentation fault

    // -----------------------------//

    // -----------------------------//
    *pi = 5;

    // imagine the above line doesnt exist
    // valgrind will create an error for every byte (4) 
    // -----------------------------//

    printf("location of pi  : %p\n", pi);
    printf("value of pi     : %d\n", *pi);

    free(pi); // step 1: free memory
    pi = NULL; // step 2: null the pointer
    // NULL the pointer so it is inacessable now. it is a safeguard and good practice


    // -------another type of error--------//
    // reading from this pointer again results in undefined behavior
    // [after freeing, not after nulling]

    // free(pi);
    // printf("location of pi : %p\n", pi);
    // printf("value of pi : %d\n", *pi);

    // ------------------------------------//

    // now imagine i reassign after freeing

    // free(pi);
    // *pi = 3;

    // printf("location of pi : %p\n", pi);
    // printf("value of pi : %d\n", *pi);

    // is again undefined behavior, and valgrind will return a memory error
    // this is called the USE AFTER FREE ERROR
    // ------------------------------------//
    // this is called the DOUBLE FREE ERROR
    // free(pi);
    // free(pi);
    // and usually cannot be compiled
    // ------------------------------------//
    
    return 0;
}


./a.out 

^^ program from memory.c

## a working game of nim:

In [ ]:
unsigned int *read_game(size_t *num_piles)
{
    fscanf(stdin, "%zu", num_piles);
    unsigned int *pile_size;
    pile_size = malloc(sizeof(unsigned int) * *num_piles); // pointer multiplication
    // read each pile size
    size_t i = 0;
    while (i < *num_piles && scanf("%u", &pile_size[i]) == 1)
        i++;
    // fill in any missing values with 0
    while (i < *num_piles)
    {
        pile_size[i] = 0;
        i++;
    }
    return pile_size;
}

note the presence of pointer multiplication:

In [ ]:
pile_size = malloc(sizeof(unsigned int) * *num_piles);

also note the lack of a free() statement

# using the heap void 
• *malloc (size_t size)
    • Allocates size bytes of uninitialized storage
• void *calloc (size_t num, size_t size)
    • Allocates memory for an array of num objects of size bytes each
    • Initializes all bytes in the allocated storage to zero
    
• Each function returns a pointer to the newly-allocated memory
    • Might return null if allocation was unsuccessful (extremely unlikely)
• void* is the type denoting “a pointer to something”
• You should almost never declare variables of type void*
    • Functions deal in void* so they are generic



### nim again

In [ ]:
unsigned int *read_game(size_t *num_piles)
{
size_t max_piles = 8;
size_t pile_count = 0;
unsigned int *pile_size = malloc(sizeof(*pile_size) * max_piles);
unsigned int pile;
while (fscanf(stdin, "%u", &pile) == 1)
{
pile_size[pile_count] = pile;
pile_count++;
}
*num_piles = pile_count;
return pile_size;
}

but what if the array is full? what if there are more than 8 values

In [ ]:
// ...
while (fscanf(stdin, "%u", &pile) == 1) {
if (max_piles == pile_count)
{
max_piles += 8;
unsigned int *bigger
= malloc(sizeof(*bigger) * max_piles);
for (size_t i = 0; i < pile_count; i++)
{
bigger[i] = pile_size[i];
}
//
// = bigger;
}
pile_size[pile_count] = pile;
pile_count++;
}

but this is a leak! you reassigned a pointer to another bigger pointer!